In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class DataLoader:
    req_columns = ['CH_TIMESTAMP', 'CH_SYMBOL', 'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE', 'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE', 'CH_TOT_TRADED_QTY', 'CH_52WEEK_HIGH_PRICE', 'CH_52WEEK_LOW_PRICE']
    new_column_names = ['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close', 'volume', 'high_52w', 'low_52w']

    @staticmethod
    def load_data(script_name, start_date, end_date, series="EQ"):
        df = equity_history(script_name, series, start_date, end_date)[DataLoader.req_columns]
        df.columns = DataLoader.new_column_names
        return df

In [121]:
def identify_marubozus(df):
    df['marubozu'] = np.where(
        np.logical_and.reduce(
            [df['candle_color'] == 'green', 
             df['candle_body_ratio'] > 0.85, 
             df['pct_change'].abs() > 0.5]
        ),
        "Bullish",
        np.where(
            np.logical_and.reduce(
                [df['candle_color'] == 'red', 
                 df['candle_body_ratio'] > 0.85, 
                 df['pct_change'].abs() > 0.5]
            ),
            "Bearish",
            None
        )
    )
    return df

def identify_dojis(df):
    df['is_doji'] = np.where(
        np.logical_or(
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.15,
                    df['candle_color'] == 'red',
                    ((df['high'] - df['open']) / df['candle_length']).between(.33, .67)
                ]
            ),
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.15,
                    df['candle_color'] == 'green',
                    ((df['high'] - df['close']) / df['candle_length']).between(.33, .67)
                ]
            ),
        ),
        True,
        False
    )
    return df

def identify_spinning_tops(df):
    df['is_spinning_top'] = np.where(
        np.logical_or(
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.30,
                    df['candle_color'] == 'red',
                    ((df['high'] - df['open']) / df['candle_length']).between(.2, .35)
                ]
            ),
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.30,
                    df['candle_color'] == 'green',
                    ((df['high'] - df['close']) / df['candle_length']).between(.2, .35)
                ]
            ),
        ),
        True,
        False
    )
    return df

def identify_paper_umbrellas(df):
    df['paper_umbrella_type'] = np.where(
        np.logical_or(
            np.logical_and.reduce([
                df['candle_body_ratio'] < 0.38,
                df['candle_color'] == 'green',
                ((df['open'] - df['low']) / df['candle_length']) >= 0.60,
                ((df['high'] - df['close']) / df['candle_length']) <= 0.125,
            ]),
            np.logical_and.reduce([
                df['candle_body_ratio'] < 0.38,
                df['candle_color'] == 'red',
                ((df['close'] - df['low']) / df['candle_length']) >= 0.60,
                ((df['high'] - df['open']) / df['candle_length']) <= 0.125,
            ])
        ),
        'Yes',
        'No'
    )
    return df

def identify_shooting_stars(df):
    df['is_shooting_star'] = np.where(
        np.logical_or(
            np.logical_and.reduce([
                df['candle_body_ratio'] < 0.38,
                df['candle_color'] == 'green',
                ((df['high'] - df['close']) / df['candle_length']) >= 0.6
            ]),
            np.logical_and.reduce([
                df['candle_body_ratio'] < 0.38,
                df['candle_color'] == 'red',
                ((df['high'] - df['open']) / df['candle_length']) >= 0.6
            ])
        ),
        True,
        False
    )
    return df

In [3]:
script_name = 'INFY'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 365)

In [4]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsep

In [5]:
df['candle_body_length'] = (df['open'] - df['close']).abs()
df['candle_length'] = df['high'] - df['low']
df['candle_body_ratio'] = df['candle_body_length'] / df['candle_length']
df['candle_color'] = np.where(df.close > df.open, 'green', 'red')
df['pct_change'] = (df['close'] - df['prev_close']) * 100 / df['prev_close']

In [119]:
for fn in [identify_marubozus, identify_dojis, identify_paper_umbrellas, identify_shooting_stars]:
    df = fn(df)

In [92]:
for i in [3, 5, 7, 14]:
    df[f'sma_{i}d'] = ((df.high + df.low) / 2).rolling(i).mean()
    df[f'trend_{i}d'] = (df[f'sma_{i}d'] - df[f'sma_{i}d'].shift(i)) / i

In [117]:
df[df.paper_umbrella_type == "Yes"] #[["date", "open", "high", "low", "close", ""]]

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,...,paper_umbrella_type,is_shooting_star,sma_3d,trend_3d,sma_5d,trend_5d,sma_7d,trend_7d,sma_14d,trend_14d
214,2021-11-08,INFY,1741.00,1703.00,1730.00,1737.50,1740.00,1708.70,5245903,1848.00,...,Yes,False,1713.741667,7.377778,1705.845,-0.135,1703.492857,-7.303571,1729.055357,2.271811
145,2022-01-06,INFY,1828.00,1800.00,1828.00,1817.80,1820.50,1844.65,6449205,1914.05,...,Yes,False,1859.258333,-11.133333,1872.735,-1.012,1877.264286,4.816837,1860.405357,8.241199
102,2022-04-13,INFY,1757.50,1732.00,1754.95,1748.55,1751.00,1742.45,6257693,1953.90,...,Yes,False,1758.550000,-22.780556,1783.300,-19.104,1804.242857,-11.754082,1845.382143,1.171811
49,2022-07-08,INFY,1516.95,1499.95,1515.00,1514.45,1515.00,1498.70,3652582,1953.90,...,Yes,False,1499.425000,7.388889,1493.275,4.880,1485.507143,4.203571,1470.794643,0.321173
10,2022-08-02,INFY,1547.00,1528.35,1546.55,1543.60,1539.90,1551.05,4374635,1953.90,...,Yes,False,1543.491667,21.500000,1519.325,3.157,1509.439286,4.218878,1494.673214,1.437372
14,2022-08-08,INFY,1622.00,1596.00,1617.00,1619.40,1617.05,1616.65,3763409,1953.90,...,Yes,False,1604.133333,18.758333,1581.005,15.010,1571.117857,10.489286,1534.405357,4.526913
27,2022-08-29,INFY,1468.55,1450.00,1468.55,1461.75,1463.50,1521.55,8079050,1953.90,...,Yes,False,1507.233333,-16.338889,1520.790,-14.488,1540.778571,-8.951020,1572.107143,3.358929


In [120]:
df[df["is_shooting_star"]] #[["date", "open", "high", "low", "close", ""]]

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,...,paper_umbrella_type,is_shooting_star,sma_3d,trend_3d,sma_5d,trend_5d,sma_7d,trend_7d,sma_14d,trend_14d
238,2021-09-22,INFY,1738.90,1713.05,1717.00,1716.90,1715.65,1718.45,4553892,1757.00,...,No,True,1708.166667,1.697222,1706.215,2.895,1703.000000,NaN,NaN,NaN
245,2021-10-01,INFY,1678.95,1661.25,1665.10,1665.15,1665.60,1675.20,5733906,1788.00,...,No,True,1678.283333,-19.047222,1694.180,-6.275,1710.596429,1.085204,1706.798214,NaN
175,2021-12-01,INFY,1741.90,1713.00,1715.00,1714.90,1717.00,1712.65,5193264,1848.00,...,No,True,1711.000000,0.666667,1709.150,-9.239,1712.660714,-8.383163,1742.001786,1.715434
189,2021-12-21,INFY,1838.00,1801.00,1801.00,1811.60,1814.70,1798.90,5353584,1848.00,...,No,True,1817.450000,23.700000,1790.945,8.248,1778.067857,5.653061,1758.282143,1.162883
146,2022-01-07,INFY,1836.00,1806.80,1815.45,1814.30,1816.00,1817.80,4834389,1914.05,...,No,True,1835.616667,-19.041667,1860.015,-4.361,1868.164286,1.953061,1861.328571,7.830102
148,2022-01-11,INFY,1870.00,1850.15,1855.00,1855.60,1860.00,1850.75,5142287,1914.05,...,No,True,1840.975000,-6.094444,1841.675,-9.881,1857.371429,-2.381122,1865.705357,7.214796
160,2022-01-28,INFY,1727.55,1679.50,1681.00,1686.20,1685.80,1678.60,10998502,1953.90,...,No,True,1703.708333,-28.994444,1730.290,-33.886,1767.517857,-17.760204,1829.678571,-2.260714
115,2022-02-14,INFY,1718.55,1678.25,1690.00,1682.20,1684.00,1721.35,7459637,1953.90,...,No,True,1729.733333,3.444444,1726.740,-4.332,1725.635714,-1.231633,1729.946429,-9.142730
82,2022-06-03,INFY,1555.00,1518.00,1524.90,1521.70,1522.75,1508.00,9395707,1953.90,...,No,True,1508.216667,6.875000,1507.045,12.888,1486.564286,2.557143,1477.614286,-5.073469
37,2022-06-22,INFY,1459.60,1435.00,1436.05,1437.25,1436.90,1449.90,6253366,1953.90,...,No,True,1429.616667,6.255556,1417.755,-7.221,1421.060714,-10.854082,1459.050000,-1.115561


In [ ]:
df[]